In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.18.0 requires google-genai<2.0.0,>=1.45.0, but you have google-genai 1.7.0 which is incompatible.
google-cloud-aiplatform 1.125.0 requires google-genai<2.0.0,>=1.37.0, but you have google-genai 1.7.0 which is incompatible.


In [2]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [4]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

#### Create a local database

In [5]:
%load_ext sql
%sql sqlite:///sample.db

#### Create and insert data into the tables

In [6]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);


 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

#### Define database functions

In [7]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

The first function will list all tables available in the database. Define it, and test it out to ensure it works.

In [8]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['products', 'sqlite_sequence', 'staff', 'orders']

Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.

In [9]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("products")

 - DB CALL: describe_table(products)


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10, 2)')]

Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a SELECT query. Now provide that functionality, and test that it works.

In [10]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from products")

 - DB CALL: execute_query(select * from products)


[(1, 'Laptop', 799.99), (2, 'Keyboard', 129.99), (3, 'Mouse', 29.99)]

In [11]:
execute_query("select * from staff")

 - DB CALL: execute_query(select * from staff)


[(1, 'Alice', 'Smith'), (2, 'Bob', 'Johnson'), (3, 'Charlie', 'Williams')]

In [12]:
execute_query("select * from orders")

 - DB CALL: execute_query(select * from orders)


[(1, 'David Lee', 1, 1), (2, 'Emily Chen', 2, 2), (3, 'Frank Brown', 1, 3)]

#### Implementing function calls

In [13]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for a computer store. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query."""

In [14]:
client = genai.Client(api_key=GOOGLE_API_KEY)

chat = client.chats.create(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools
    )
)

In [15]:
response = chat.send_message("What is the cheapest product?")
print(f"\n{response.text}")

 - DB CALL: execute_query(SELECT * FROM Products ORDER BY Price ASC LIMIT 1;)

The cheapest product is the Mouse, which costs $29.99.



In [16]:
# something more complex
response = chat.send_message("What product did Emily Chen buy, what was the name of the person who sold it to her and what was the price?")
print(f"\n{response.text}")

 - DB CALL: execute_query(SELECT p.Name, e.FirstName, e.LastName, p.Price FROM Sales s JOIN Products p ON s.ProductID = p.ProductID JOIN Employees e ON s.EmployeeID = e.EmployeeID JOIN Customers c ON s.CustomerID = c.CustomerID WHERE c.FirstName = 'Emily' AND c.LastName = 'Chen';)

I am sorry, I cannot answer this question as I do not have access to the Sales table.


In [17]:
response = chat.send_message('What products should salesperson Alice focus on to round out her portfolio? Explain why.')
print(f"\n{response.text}")

 - DB CALL: list_tables()
 - DB CALL: describe_table(products)
 - DB CALL: describe_table(staff)
 - DB CALL: describe_table(orders)
 - DB CALL: execute_query(SELECT p.product_name FROM products p LEFT JOIN orders o ON p.product_id = o.product_id LEFT JOIN staff s ON o.staff_id = s.staff_id WHERE s.first_name = 'Alice' GROUP BY p.product_name ORDER BY COUNT(o.order_id) ASC LIMIT 3)

Based on the data, Alice should focus on selling more Laptops and Mice. These are the products that she has sold the least of, so increasing her sales in these areas would help to round out her portfolio.



##### Inspecting the conversation

In [18]:
import textwrap


def print_chat_turns(chat):
    """Prints out each turn in the chat history, including function calls and responses."""
    for event in chat.get_history():
        print(f"{event.role.capitalize()}:")

        for part in event.parts:
            if txt := part.text:
                print(f'  "{txt}"')
            elif fn := part.function_call:
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Function call: {fn.name}({args})")
            elif resp := part.function_response:
                print("  Function response:")
                #print(textwrap.indent(str(resp.response['result']), "    "))
                response_data = resp.response
                result_data = response_data.get('result', 'No result key found in response.')
                print(textwrap.indent(str(result_data), "    "))

        print()


print_chat_turns(chat)

User:
  "What is the cheapest product?"

Model:
  Function call: execute_query(sql=SELECT * FROM Products ORDER BY Price ASC LIMIT 1;)

User:
  Function response:
    [(3, 'Mouse', 29.99)]

Model:
  "The cheapest product is the Mouse, which costs $29.99.
"

User:
  "What product did Emily Chen buy, what was the name of the person who sold it to her and what was the price?"

Model:
  Function call: execute_query(sql=SELECT p.Name, e.FirstName, e.LastName, p.Price FROM Sales s JOIN Products p ON s.ProductID = p.ProductID JOIN Employees e ON s.EmployeeID = e.EmployeeID JOIN Customers c ON s.CustomerID = c.CustomerID WHERE c.FirstName = 'Emily' AND c.LastName = 'Chen';)

User:
  Function response:
    No result key found in response.

Model:
  "I am sorry, I cannot answer this question as I do not have access to the Sales table."

User:
  "What products should salesperson Alice focus on to round out her portfolio? Explain why."

Model:
  Function call: list_tables()

User:
  Function respo

### Compositional function calling

In [19]:
from pprint import pformat
from IPython.display import display, Image, Markdown

async def handle_response(stream, tool_impl=None):
  """Stream output and handle any tool calls during the session."""
  all_responses = []

  async for msg in stream.receive():
    all_responses.append(msg)

    if text := msg.text:
      # Output any text chunks that are streamed back.
      if len(all_responses) < 2 or not all_responses[-2].text:
        # Display a header if this is the first text chunk.
        display(Markdown('### Text'))

      print(text, end='')

    elif tool_call := msg.tool_call:
      # Handle tool-call requests.
      for fc in tool_call.function_calls:
        display(Markdown('### Tool call'))

        # Execute the tool and collect the result to return to the model.
        if callable(tool_impl):
          try:
            result = tool_impl(**fc.args)
          except Exception as e:
            result = str(e)
        else:
          result = 'ok'

        tool_response = types.LiveClientToolResponse(
            function_responses=[types.FunctionResponse(
                name=fc.name,
                id=fc.id,
                response={'result': result},
            )]
        )
        await stream.send(input=tool_response)

    elif msg.server_content and msg.server_content.model_turn:
      # Print any messages showing code the model generated and ran.

      for part in msg.server_content.model_turn.parts:
          if code := part.executable_code:
            display(Markdown(
                f'### Code\n```\n{code.code}\n```'))

          elif result := part.code_execution_result:
            display(Markdown(f'### Result: {result.outcome}\n'
                             f'```\n{pformat(result.output)}\n```'))

          elif img := part.inline_data:
            display(Image(img.data))

  print()
  return all_responses

In [23]:
model = 'gemini-2.0-flash-exp'
live_client = genai.Client(api_key=GOOGLE_API_KEY,
                           http_options=types.HttpOptions(api_version='v1alpha'))

# Wrap the existing execute_query tool you used in the earlier example.
execute_query_tool_def = types.FunctionDeclaration.from_callable(
    client=live_client, callable=execute_query)

# Provide the model with enough information to use the tool, such as describing
# the database so it understands which SQL syntax to use.
sys_int = """You are a database interface.  
Whenever the user mentions a table, ALWAYS retrieve the schema automatically  
using the execute_query tool with a PRAGMA table_info(table_name) query  
BEFORE attempting any inserts, updates, or synthetic data generation.  

NEVER ask the user for the schema. Instead, query it directly.
"""

config = {
    "response_modalities": ["TEXT"],
    "system_instruction": {"parts": [{"text": sys_int}]},
    "tools": [
        {"code_execution": {}},
        {"function_declarations": [execute_query_tool_def.to_json_dict()]},
    ],
}

async with live_client.aio.live.connect(model=model, config=config) as session:

  message = "Can you generate and insert 5 new rows in the orders table."
  print(f"> {message}\n")

  await session.send(input=message, end_of_turn=True)
  await handle_response(session, tool_impl=execute_query)

> Can you generate and insert 5 new rows in the orders table.



### Text

Okay, I can help you with that. First, I need to know the schema of the `orders` table.


### Code
```
sql = "PRAGMA table_info(orders)"
print(sql)

```

### Result: Outcome.OUTCOME_OK
```
'PRAGMA table_info(orders)\n'
```

...

### Tool call

 - DB CALL: execute_query(
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
('John Doe', 1, 101),
('Jane Smith', 2, 102),
('Peter Jones', 3, 103),
('Mary Brown', 1, 104),
('David Wilson', 2, 105);
)


### Result: Outcome.OUTCOME_OK
```
"{'result': []}\n"
```

### Text

The rows have been inserted successfully.



In [21]:
async with live_client.aio.live.connect(model=model, config=config) as session:

  message = "Can you figure out the number of orders that were made by each of the staff?"

  print(f"> {message}\n")
  await session.send(input=message, end_of_turn=True)
  await handle_response(session, tool_impl=execute_query)

  message = "Generate and run some code to plot this as a python seaborn chart"

  print(f"> {message}\n")
  await session.send(input=message, end_of_turn=True)
  await handle_response(session, tool_impl=execute_query)

> Can you figure out the number of orders that were made by each of the staff?



### Code
```
default_api.execute_query(sql="SELECT staff_id, COUNT(order_id) FROM Orders GROUP BY staff_id")

```

### Tool call

 - DB CALL: execute_query(SELECT staff_id, COUNT(order_id) FROM Orders GROUP BY staff_id)


### Text

...

### Tool call

 - DB CALL: execute_query(
SELECT s.staff_id, COUNT(o.order_id) AS order_count
FROM Staff s
JOIN Orders o ON s.staff_id = o.staff_id
GROUP BY s.staff_id;
)


### Result: Outcome.OUTCOME_OK
```
"{'result': [[1, 2], [2, 1]]}\n"
```

### Text

It appears the data in the tables is too limited to give a meaningful output. If there was more data this could easily be plotted with seaborn.
The data is as follows:

Staff ID 1: 2 orders
Staff ID 2: 1 order

To plot this with Seaborn in Python, you would do the following:

```python
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

data = {'Staff ID': [1, 2], 'Order Count': [2, 1]}
df = pd.DataFrame(data)

sns.barplot(x='Staff ID', y='Order Count', data=df)
plt.title('Number of Orders per Staff Member')
plt.show()
```

